# FFMPEG
## Piping numpy arrays into FFMPEG


In [ ]:
import subprocess as sp
command = [ 'ffmpeg', 
           '-f', 'rawvideo', '-vcodec','rawvideo', 
           '-s', '1920x1080', # size of one frame 
           '-pix_fmt', 'rgb24', 
           '-r', '24', # frames per second 
           '-i', '-', # The imput comes from a pipe 
           '-an', # Tells FFMPEG not to expect any audio 
           '-vcodec', 'libx265', 
           'my_output_videofile.mp4' ]

pipe = sp.Popen( command, stdin=sp.PIPE, stderr=sp.PIPE)

# pipe = sp.Popen(command, stdin=sp.PIPE,stderr=sp.PIPE)
while (cap.isOpened()):
ret, frame = cap.read()
# Do some stuff
pipe.proc.stdin.write( my_image_array.tostring() )

http://zulko.github.io/blog/2013/09/27/read-and-write-video-frames-in-python-using-ffmpeg/

In [ ]:
import subprocess as sp
import numpy as np

def get_frame(file, time="00:00:00", shape=(1920, 1080, 3)):
    command = [ 'ffmpeg', 
               '-ss', time, 
               '-i', file, 
               '-f', 'image2pipe', 
               '-pix_fmt', 'rgb24', 
               '-vcodec', 'rawvideo', '-']
    pipe = sp.Popen(command, stdout=sp.PIPE, bufsize=10**8)
    width = shape[0]
    height = shape[1]
    depth = shape[2]
    frame_size = width * height * depth
    raw_image = pipe.stdout.read(frame_size)
    # transform the byte read into a numpy array
    image =  numpy.frombuffer(raw_image, dtype='uint8')
    image = image.reshape((height, width, depth))

    # throw away the data in the pipe's buffer.
    pipe.stdout.flush()
    return image

In [ ]:
from PIL import Image

file = "../data/videos/video.mp4"
time = ["00:10:00", "00:20:00", "00:30:00"]#, "02:00:00"]


for i, t in enumerate(time):
    frame_image = get_frame(file, time=t, shape=(1920, 1080, 3))
    Image.fromarray(frame_image).save("../data/images/video_img/video_{}.tif".format(i), format="tiff")

In [ ]:
import subprocess as sp

FFMPEG_BIN = "ffmpeg"
# FFMPEG_BIN = "ffmpeg.exe"
video = "../data/videos/video.mp4"
# bufsize = 1920 * 800 * 3 * 8
command = [ FFMPEG_BIN, 
           '-ss', '01:00:00', 
           '-i', video, 
           #'-ss', '1', 
           '-f', 'image2pipe', 
           '-pix_fmt', 'rgb24', 
           '-vcodec', 'rawvideo', '-']
pipe = sp.Popen(command, stdout = sp.PIPE, bufsize=10**8)

In [ ]:
import numpy
# read 420*360*3 bytes (= 1 frame)
frame_width = 1920
frame_height = 800
frame_depth = 3
frame_size = frame_width * frame_height * frame_depth
raw_image = pipe.stdout.read(frame_size)
# transform the byte read into a numpy array
image_string =  numpy.fromstring(raw_image, dtype='uint8')
image_buffer =  numpy.frombuffer(raw_image, dtype='uint8')

image_string = image_string.reshape((frame_height, frame_width, frame_depth))
image_buffer = image_buffer.reshape((frame_height, frame_width, frame_depth))

# throw away the data in the pipe's buffer.
pipe.stdout.flush()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

def showframe(frame):
    #rows, cols, depth = frame.shape
    #extent = (0, cols, rows, 0)
    fig, ax = plt.subplots()
    ax.imshow(frame)#, extent=extent)
    #ax.set_axis_off()
    #fig.tight_layout(pad=0)
    fig.show()

showframe(image_string)
showframe(image_buffer)
Image.fromarray(image_buffer).save("../data/images/video_img/video_2.tif", format="tiff")

You can also get informations on a file (frames size, number of frames per second, etc.) by calling:

In [ ]:
FFMPEG_BIN = "ffmpeg"
video = "../data/videos/video.mp4"

command = [FFMPEG_BIN, '-i', video, '-']
pipe = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE)
# with sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE) as pipe:
while pipe.stderr:
    info = pipe.stderr.readline()
    print(info)

    #info_stdout = pipe.stdout.readline()
    #print("Info: {}".format(info_stderr))

# info_stdout = pipe.stdout.readline()
# info_stderr = pipe.stderr.readline()
# pipe.terminate()
# infos = proc.stderr.read()

# Convert .MOV to .MP4

The command to just stream it to a new container (mp4) needed by some applications like premiere without encoding (fast) is:

In [ ]:
ffmpeg -i movie.mov -vcodec copy -acodec copy out.mp4

Alternative as mentioned in the comments, which re-encodes with best quaility (-qscale 0):

In [ ]:
ffmpeg -i movie.mov -qscale 0 output.mp4